# 📓 Groundedness Evaluations for Abstention Handling

In many ways, feedbacks can be thought of as LLM apps themselves. Given text, they return some result. Thinking in this way, we can use TruLens to evaluate and track our feedback quality. We can even do this for different models or prompting schemes (such as chain-of-thought reasoning).

This notebook follows an evaluation of a set of test cases generated from human annotated datasets. In particular, we generate test cases from [SummEval](https://arxiv.org/abs/2007.12626).

SummEval is one of the datasets dedicated to automated evaluations on summarization tasks, which are closely related to the groundedness evaluation in RAG with the retrieved context (i.e. the source) and response (i.e. the summary). It contains human annotation of numerical score (**1** to **5**) comprised of scoring from 3 human expert annotators and 5 croweded-sourced annotators. There are 16 models being used for generation in total for 100 paragraphs in the test set, so there are a total of 16,000 machine-generated summaries. Each paragraph also has several human-written summaries for comparative analysis. 

For evaluating groundedness feedback functions, we compute the annotated "consistency" scores, a measure of whether the summarized response is factually consisntent with the source texts and hence can be used as a proxy to evaluate groundedness in our RAG triad, and normalized to **0** to **1** score as our **expected_score** and to match the output of feedback functions.

## Abstention Background

In this particular set of evaluations, we are focused on the handling of abstentions. Uncertainty-based abstention in LLMs has been shown to improve safety and reduce hallucination ([Tomani](https://arxiv.org/abs/2404.10960)). For groundedness evaluations, we want to ensure these are handled in a manner that is consistent with human preferences; in other words, calibrated.

Abstentions can be broken down into two distinct groups, distinguished by whether the question is answerable given the context. In other words, whether the abstention is **justified**. We take an opinionated stance that abstentions for unanswerable questions are justified and therefore, **grounded**. Alternatively, abstentions for questions answerable by the context are therefore not grounded.

## Experimental Setup

For this set of experiments, we take the same test cases used for groundedness evaluations with a few key changes:
1. We randomly sample approximately 50% of the test cases and replace the response with an abstention. We'll refer to this as the abstention set. The rest will be head as control.
2. In the abstention set, randomly sample approximately 50% of the test cases to generate a related question using GPT4o-mini, and generate an unrelated question for the rest. This splits the absention set into **answerable** and **unanswerable** abstentions.

From here, we have two tests sets to test against.
1. The expected score for all abstentions is 1.
2. The expected score for answerable abstentions will be set to 0, and then the expected score for unanswerable abstentions will be set to 1.

We will then compute the MAE of our groundedness evaluator against the expected score for each test set. Results will be displayed for the following subgroups:
- Abstentions v. Control
- Answerable Abstentions v. Unanswerable Abstentions

We will also show results for the test cases with high and low human-annotated consistency to ensure that this treatment is consistent across expected scores.


## Improving Groundedness

To improve the groundedness feedback function against these tests, we consider the following changes:
- Abstentions are excluded from statements to evaluate; empty strings are considered grounded.
- The answerability in abstention cases is assessed; unanswerable abstentions score high, answerable abstentions score low.

In [1]:
# Import groundedness feedback function
from trulens_eval.feedback import GroundTruthAgreement
from trulens_eval import TruBasicApp, Feedback, Tru, Select
from test_cases import generate_summeval_groundedness_golden_set
import copy

tru = Tru()

# generator for groundedness golden set
test_cases_gen = generate_summeval_groundedness_golden_set("./datasets/summeval/summeval_test_100.json")

/opt/anaconda3/envs/snowday/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [2]:
# Specify the number of test cases we want to run the smoke test on
groundedness_golden_set_abstentions = []
for i in range(500):
    groundedness_golden_set_abstentions.append(next(test_cases_gen))

In [3]:
import random
from openai import OpenAI

client = OpenAI()

# Specify the number of test cases we want to run the smoke test on
groundedness_golden_set_abstentions = []
for i in range(500):
    groundedness_golden_set_abstentions.append(next(test_cases_gen))

# Randomly sample 50% of the test cases and replace the response with a random abstention
# Split into answerable and unanswerable set
abstention_set = []
control_set = []
for item in groundedness_golden_set_abstentions:
    if random.random() < 0.5:
        abstention_set.append(item)
    else:
        control_set.append(item)

for item in abstention_set:
    item['response'] = random.choice([
        "I don't know",
        "I don't have the information available to answer this question",
        "I'm unsure of the answer given the provided context",
        "I'm not certain",
        "I can't say for sure",
        "I'm not familiar with that",
        "I'm not knowledgeable about this topic",
        "I'm not the right person to ask",
        "I'm unable to provide a definitive answer",
        "I'm sorry, I can't help with that",
        "This is beyond my current understanding",
        "That's outside my area of expertise",
        "I need more information to give a proper response",
        "This topic is a bit too complex for a quick answer",
        "I'm not equipped to answer this question",
        "My knowledge on this subject is limited",
        "I don't have a clear answer for you",
        "This requires more research than I can provide at the moment",
        "I'm not confident enough in my knowledge to give an answer",
        "I'm afraid I don't have an answer to that",
        "This question stumps me, sorry",
        "I'm drawing a blank on this one",
        "I'm not sure I'm the best source for this information",
        "I don't have the expertise to answer this",
        "I'm at a loss for words on this topic",
        "I'm not sure how to respond to that",
        "I don't have a definitive answer",
        "I'm not certain how to answer",
        "I'm not sure I understand the question fully",
        "I'm not able to provide a satisfactory answer",
        "I'm not sure I can provide the best answer",
        "I'm not qualified to answer this question",
        "I'm not sure I have the information you're looking for",
        "I'm not able to answer that question",
        "I'm not sure I can be of help with that question",
        "I'm not the right one to answer that",
        "I'm not able to give a precise answer",
        "I'm not sure I have a good answer for you",
        "I'm not able to provide an accurate answer",
        "I'm not sure I can answer that accurately",
        "I'm not able to give you a clear answer",
        "I'm not sure I can be of assistance with that question",
        "I'm not able to provide a clear response",
        "I'm not sure I can give you the answer you're looking for",
        "I'm not able to provide the information you need",
        "I'm not sure I can help with that",
        "I'm not able to give a definitive response",
        "I'm not sure I can provide a clear answer",
        "I'm not able to answer that with certainty",
        "I'm not sure I have the answer to that",
        "I'm not able to provide a definite answer",
        "I'm not sure I can give a precise answer",
        "I'm not able to give an exact answer",
        "I'm not sure I can provide an exact response",
        "I'm not able to provide a specific answer",
        "I'm not sure I can offer a specific response",
        "I'm not able to give a detailed answer",
        "I'm not sure I can provide a detailed response",
        "I'm not able to offer a comprehensive answer",
        "I'm not sure I can give a comprehensive response",
        "I'm not able to provide a thorough answer",
        "I'm not sure I can offer a thorough response",
        "I'm not able to give an in-depth answer",
        "I'm not sure I can provide an in-depth response",
        "I'm not able to offer a nuanced answer",
        "I'm not sure I can give a nuanced response",
        "I'm not able to provide a nuanced explanation",
        "I'm not sure I can offer a nuanced explanation",
        "I'm not able to give a complete answer",
        "I'm not sure I can provide a complete response",
        "I'm not able to offer a complete explanation",
        "I'm not sure I can give a complete explanation",
        "I'm not able to provide a full answer",
        "I'm not sure I can offer a full response",
        "I'm not able to give a full explanation",
        "I'm not sure I can provide a full explanation",
        "I'm not able to offer a detailed explanation",
        "I'm not sure I can give a detailed explanation",
        "I'm not able to provide an exhaustive answer",
        "I'm not sure I can offer an exhaustive response",
        "I'm not able to give an exhaustive explanation",
        "I'm not sure I can provide an exhaustive explanation",
        "I'm not able to offer a precise explanation",
        "I'm not sure I can give a precise explanation",
        "I'm not able to provide a specific explanation",
        "I'm not sure I can offer a specific explanation",
        "I'm not able to give a clear explanation",
        "I'm not sure I can provide a clear explanation",
        "I'm sorry, but I can't provide the information requested",
        "Unfortunately, I'm not able to answer that question",
        "I'm unable to assist with this query",
        "My capabilities do not extend to providing an answer for this",
        "I'm sorry, but that's beyond what I can offer assistance with",
        "I don't have the capability to answer that question",
        "That's outside the scope of my assistance",
        "I'm not equipped to provide an answer to that question",
        "I'm sorry, but I can't help with that specific request",
        "Unfortunately, I don't have the resources to answer that",
        "I'm unable to provide a response to that question",
        ])
    item['group'] = 'Abstention'
    split_result = item['query'].split('.', 1)
    item['user_question'] = split_result[0] # keep the original first part of the query as teh user queestion no matter what

# Set the group for the remaining cases as 'Control'
for item in groundedness_golden_set_abstentions:
    if 'group' not in item:
        item['group'] = 'Control'
        item['user_question'] = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Write a question that can be answered from the following information: {item['query']}"},
            ]
            ).choices[0].message.content

# Split into answerable and unanswerable set
unanswerable_abstentions = []
answerable_abstentions = []

abstention_set_copy = copy.deepcopy(abstention_set)
for item in abstention_set_copy:
    item['expected_score'] = 1.0
    if random.random() < 0.5:
        item['group'] = 'Unanswerable Abstention'
        item['user_question'] = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Write a random, fact-based question. It can be about anything."},
            ]
            ).choices[0].message.content
        unanswerable_abstentions.append(item)
    else:
        item['group'] = 'Answerable Abstention'
        item['user_question'] = random.choice([
                f"What is the answer to this question: {split_result[-1]}?",
                f"Can you answer this question: {split_result[-1]}?"
        ])
        item['user_question'] = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Write a question that can be answered from the following information: {item['query']}"},
            ]
            ).choices[0].message.content
        answerable_abstentions.append(item)
    
# create a set that includes all abstentions and the control set
groundedness_golden_set_abstensions_score_high = control_set + answerable_abstentions + unanswerable_abstentions

In [4]:
import pickle

# Write out the data
with open('groundedness_golden_set_abstensions.pkl', 'wb') as file:
    pickle.dump(groundedness_golden_set_abstensions_score_high, file)

In [5]:
# Load the data back in
with open('groundedness_golden_set_abstensions.pkl', 'rb') as file:
    groundedness_golden_set_abstensions_score_high = pickle.load(file)

In [7]:
answerable_abstentions_ca = copy.deepcopy(answerable_abstentions)
for item in answerable_abstentions_ca:
    item['expected_score'] = 0.0

groundedness_golden_set_abstensions_consider_answerability = control_set + answerable_abstentions_ca + unanswerable_abstentions

In [8]:
groundedness_golden_set_abstensions_consider_answerability_copy = copy.deepcopy(groundedness_golden_set_abstensions_consider_answerability)

test_set = [{'query': item['query'], 'response': item['response'], 'user_question': item['user_question'], 'group': item['group']} for item in groundedness_golden_set_abstensions_consider_answerability_copy]

In [9]:
import pandas as pd

# Consider answerability of abstentions
print("\n Consider answerability of abstentions")

## Count the data by group (Control, Answerable Abstention, Unanswerable Abstention)
ca_df = pd.DataFrame(groundedness_golden_set_abstensions_consider_answerability)
ca_group_counts = ca_df['group'].value_counts()
print(ca_group_counts)

## Calculate average values for expected_score and human_score by group
ca_group_avg = ca_df.groupby('group').agg({'expected_score': 'mean', 'human_score': 'mean'})
ca_group_avg = ca_group_avg.reindex(['Control', 'Answerable Abstention', 'Unanswerable Abstention'])
print(ca_group_avg)

# Reward all abstentions equally
print("\n Reward all abstentions equally")

sh_df = pd.DataFrame(groundedness_golden_set_abstensions_score_high)
sh_group_counts = sh_df['group'].value_counts()
print(sh_group_counts)

## Calculate average values for expected_score and human_score by group
sh_group_avg = sh_df.groupby('group').agg({'expected_score': 'mean', 'human_score': 'mean'})
sh_group_avg = sh_group_avg.reindex(['Control', 'Answerable Abstention', 'Unanswerable Abstention'])
print(sh_group_avg)


 Consider answerability of abstentions
group
Control                    234
Unanswerable Abstention    141
Answerable Abstention      125
Name: count, dtype: int64
                         expected_score  human_score
group                                               
Control                        0.913718     4.653846
Answerable Abstention          0.000000     4.741333
Unanswerable Abstention        1.000000     4.723404

 Reward all abstentions equally
group
Control                    234
Unanswerable Abstention    141
Answerable Abstention      125
Name: count, dtype: int64
                         expected_score  human_score
group                                               
Control                        0.913718     4.653846
Answerable Abstention          1.000000     4.741333
Unanswerable Abstention        1.000000     4.723404


### Benchmarking GPT4o

In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
from trulens_eval.feedback.provider import OpenAI

openai_provider = OpenAI(model_engine="gpt-4o")
f_groundedness_openai_gpt4o = Feedback(openai_provider.groundedness_measure_with_cot_reasons,
                                       name = "Groundedness OpenAI GPT-4o - Reward Abstention")
def wrapped_groundedness_openai_gpt4o(input: str, output: str) -> float:
    return f_groundedness_openai_gpt4o(input, output)[0]

f_groundedness_openai_gpt4o_consider_answerability = Feedback(openai_provider.groundedness_measure_with_cot_reasons_consider_answerability)

def wrapped_groundedness_openai_gpt4o_consider_answerability(input: str, output: str, question: str) -> float:
    return f_groundedness_openai_gpt4o_consider_answerability(input, output, question)[0]

# Create a Feedback object using the numeric_difference method of the ground_truth object
ground_truth_consider_answerability = GroundTruthAgreement(groundedness_golden_set_abstensions_consider_answerability)

# Call the numeric_difference method with app and record and aggregate to get the mean absolute error
f_mae_consider_answerability = Feedback(ground_truth_consider_answerability.mae,
                                        name = "Mean Absolute Error (consider answerability)", higher_is_better=False).on(Select.Record.app._call.args.args[0]).on(Select.Record.app._call.args.args[1]).on(Select.RecordOutput)

# Create a Feedback object using the numeric_difference method of the ground_truth object
ground_truth_abstensions_score_high = GroundTruthAgreement(groundedness_golden_set_abstensions_score_high)
# Call the numeric_difference method with app and record and aggregate to get the mean absolute error
f_mae_abstensions_score_high = Feedback(ground_truth_abstensions_score_high.mae,
                                        name = "Mean Absolute Error (all abstensions score high)", higher_is_better=False).on(Select.Record.app._call.args.args[0]).on(Select.Record.app._call.args.args[1]).on(Select.RecordOutput)

✅ In Mean Absolute Error (consider answerability), input prompt will be set to __record__.app._call.args.args[0] .
✅ In Mean Absolute Error (consider answerability), input response will be set to __record__.app._call.args.args[1] .
✅ In Mean Absolute Error (consider answerability), input score will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Mean Absolute Error (all abstensions score high), input prompt will be set to __record__.app._call.args.args[0] .
✅ In Mean Absolute Error (all abstensions score high), input response will be set to __record__.app._call.args.args[1] .
✅ In Mean Absolute Error (all abstensions score high), input score will be set to __record__.main_output or `Select.RecordOutput` .


In [12]:
query = "A man has been arrested in connection with the death of a transgender escort  who was found strangled and beaten last month. Vanessa Santillan's body was found in a £400,000 flat in Fulham, south west London, at the end of March. The 33-year-old Mexican national, who worked as a transgender escort, died as a result of injuries to the head and neck. Vanessa Santillan, 33, was killed at the end of March. Her body was found in a flat in Fulham having been strangled and beaten to death A 23-year-old man was arrested in connection with her death but has been bailed. He has been told to return to a London police station for further questioning at a later date. Meanwhile Scotland Yard is appealing to anyone who may have had contact with Miss Santillan in the hours before her death. According to her website Miss Santillan worked in London, Paris and Miami as an escort. Police did not confirm whether her profession was central to the investigation but insisted they would do 'everything' to solve the case. London Ambulance Service was called to a flat in Romily Court, Fulham, on March 28 at around 9.30pm. Miss Santillan was pronounced dead at the scene having suffered injuries to her head and neck. The woman had been working as a transgender escort, her website revealed. Miss Santillan is understood to have moved to London from Mexico The woman, who described herself as visiting London from Miami, was pronounced dead at the scene last month Miss Santillan spoke of her love for London and Paris online in the weeks before her death. Police are urging anyone who had contact with her in the hours before her death to come forward Detective Chief Inspector Rebecca Reeves, who leads the investigation, said: 'We want to speak to anyone who saw Vanessa on Friday or Saturday. 'We need to know why this has happened and we want help from anyone who knew her while she was in London.' In the weeks before her death Miss Santillan took to social media to talk of her love for London. On her website she described herself as visiting from Miami in search of 'upscale' gentlemen. Miss Santillan's body was found when London Ambulance Service was called to an address in Fulham, south west London Sorry we are not currently accepting comments on this article."
filtered_queries = [item for item in groundedness_golden_set_abstensions_consider_answerability if item['query'] == query]
filtered_queries = [item for item in filtered_queries if item['response'] == "I'm not knowledgeable about this topic"]
filtered_queries

[{'query': "A man has been arrested in connection with the death of a transgender escort \xa0who was found strangled and beaten last month. Vanessa Santillan's body was found in a £400,000 flat in Fulham, south west London, at the end of March. The 33-year-old Mexican national, who worked as a transgender escort, died as a result of injuries to the head and neck. Vanessa Santillan, 33, was killed at the end of March. Her body was found in a flat in Fulham having been strangled and beaten to death A 23-year-old man was arrested in connection with her death but has been bailed. He has been told to return to a London police station for further questioning at a later date. Meanwhile Scotland Yard is appealing to anyone who may have had contact with Miss Santillan in the hours before her death. According to her website Miss Santillan worked in London, Paris and Miami as an escort. Police did not confirm whether her profession was central to the investigation but insisted they would do 'ever

In [15]:
tru_wrapped_groundedness_gpt4o = TruBasicApp(wrapped_groundedness_openai_gpt4o_consider_answerability,
                                             app_id="groundedness GPT-4o-instruct - Consider Answerability - test 2",
                                             feedbacks=[f_mae_consider_answerability, f_mae_abstensions_score_high])

for i in range(len(groundedness_golden_set_abstensions_score_high)):
    source = groundedness_golden_set_abstensions_consider_answerability[i]["query"]
    response = groundedness_golden_set_abstensions_consider_answerability[i]["response"]
    question = groundedness_golden_set_abstensions_consider_answerability[i]["user_question"]
    group = groundedness_golden_set_abstensions_consider_answerability[i]["group"]
   
    with tru_wrapped_groundedness_gpt4o as recording:
        try:
            recording.record_metadata = dict(group = group)
            tru_wrapped_groundedness_gpt4o.app(source, response, question)
            
        except Exception as e:
            print(e)

Unsure what the main input string is for the call to _call with args ["Esteban Cambiasso has won all the major European competitions a player can during his illustrious career but revealed that keeping Leicester City in the Premier League would be up there with the best. The Foxes are currently seven points adrift at the bottom of the table, with only eight games remaining, knowing that time is running out to save themselves. Cambiasso refuses to give up and admits that keeping Leicester up will feel like winning a trophy. Esteban Cambiasso says that helping keep Leicester in the Premier League will feel like winning a trophy 'For me, it's like another cup,' he told BBC East Midlands Today. 'When you start another season you have an objective, and this is the objective for us. 'For me, winning a cup or winning the league with another team is the same now as having the possibility to save Leicester in the Premier League.' The Argentinian midfielder poses with the trophy after his team w

In [ ]:
from trulens_eval.feedback.provider import OpenAI

openai_provider = OpenAI(model_engine="gpt-4o")

/opt/anaconda3/envs/snowday/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
openai_provider.groundedness_measure_with_cot_reasons_consider_answerability("328.2 million people live in the United States", "I don't know", "How many people live in the United States?")

hypothesis I don't know
abstention_score 1.0
Abstention detected
answerable abstention detected


(0.0, {'reasons': 'STATEMENT 0:\nAnswerable abstention\n'})

In [ ]:
openai_provider.groundedness_measure_with_cot_reasons_consider_answerability("10 million people live in Kenya", "I don't know", "How many people live in the United States?")

hypothesis I don't know
abstention_score 1.0
Abstention detected
unanswerable abstention detected


(1.0, {'reasons': 'STATEMENT 0:\nUnanswerable abstention\n'})

In [ ]:
tru_wrapped_groundedness_gpt4o = TruBasicApp(wrapped_groundedness_openai_gpt4o_consider_answerability, app_id="groundedness GPT-4o-instruct - Consider Answerability - test 2",
                                             feedbacks=[f_mae_consider_answerability, f_mae_abstensions_score_high])

for i in range(len(groundedness_golden_set_abstensions_score_high)):
    source = groundedness_golden_set_abstensions_consider_answerability[i]["query"]
    response = groundedness_golden_set_abstensions_consider_answerability[i]["response"]
    question = groundedness_golden_set_abstensions_consider_answerability[i]["user_question"]
    group = groundedness_golden_set_abstensions_consider_answerability[i]["group"]
   
    with tru_wrapped_groundedness_gpt4o as recording:
        try:
            recording.record_metadata = dict(group = group)
            tru_wrapped_groundedness_gpt4o.app(source, response, question)
            
        except Exception as e:
            print(e)


In [16]:
tru.get_leaderboard(group_by_metadata_key = "group").sort_values(by=["app_id", "group"])

Mean Absolute Error (all abstensions score high)  \
app_id                                             group                                                                       
groundedness GPT-4o-instruct                       Answerable Abstention                                            1.000000   
                                                   Control                                                          0.076120   
                                                   Unanswerable Abstention                                          1.000000   
groundedness GPT-4o-instruct - Consider Answera... Answerable Abstention                                            0.971014   
                                                   Control                                                          0.086904   
                                                   Unanswerable Abstention                                          0.000000   
groundedness GPT-4o-instruct - Consider Answera... Answerable Abstention                                            0.984000   
                                                   Control                                                          0.075940   
                                                   Unanswerable Abstention                                          0.000000   
groundedness GPT-4o-instruct - Reward Abstention   Answerable Abstention                                            0.000000   
                                                   Control                                                          0.071833   
                                                   Unanswerable Abstention                                          0.000000   

                                                                            Mean Absolute Error (consider answerability)  \
app_id                                             group                                                                   
groundedness GPT-4o-instruct                       Answerable Abstention                                        0.000000   
                                                   Control                                                      0.076120   
                                                   Unanswerable Abstention                                      1.000000   
groundedness GPT-4o-instruct - Consider Answera... Answerable Abstention                                        0.028986   
                                                   Control                                                      0.086904   
                                                   Unanswerable Abstention                                      0.401460   
groundedness GPT-4o-instruct - Consider Answera... Answerable Abstention                                        0.016000   
                                                   Control                                                      0.075940   
                                                   Unanswerable Abstention                                      0.028369   
groundedness GPT-4o-instruct - Reward Abstention   Answerable Abstention                                        1.000000   
                                                   Control                                                      0.071833   
                                                   Unanswerable Abstention                                      0.000000   

                                                                             latency  \
app_id                                             group                               
groundedness GPT-4o-instruct                       Answerable Abstention    0.295652   
                                                   Control                  2.445312   
                                                   Unanswerable Abstention  0.255814   
groundedness GPT-4o-instruct - Consider Answera... Answerable Abstention    0.579710   
                  

In [13]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.206:49509 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>